In [53]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [54]:
data_path = "./Data.csv"
df = pd.read_csv(data_path)

In [55]:
# Cheatカラムを削除して、各レコードの合計を計算する
row_sums = df.drop('Cheat', axis=1).sum(axis=1)

# 10,000をその行の合計値で割った値を計算して保持する
divisors = 1000 / row_sums

# それぞれの行に対して計算した値を掛ける
df_scaled = df.drop('Cheat', axis=1).mul(divisors, axis=0)

# データフレームにCheatカラムを追加する
df_scaled['Cheat'] = df['Cheat']
print(df_scaled)

# 新しいデータベースとして保存する
# df_scaled.to_csv('ScaledData.csv', index=False)

          karma  networkExp  bedwars_beds  bedwars_bedwars_challenger  \
0     17.016155  829.020638      0.612407                    0.000000   
1      0.065540  818.905750      0.168948                    0.000000   
2     68.870497  883.418983      0.057247                    0.000000   
3     16.078880  785.276580      0.653757                    0.000000   
4      0.000000  689.785224      0.104293                    0.000000   
...         ...         ...           ...                         ...   
4673   1.584940  403.840826      0.139218                    0.003259   
4674   0.062478  615.831959      0.089969                    0.000000   
4675   0.405830  556.337719      0.023673                    0.000000   
4676   0.140081  559.878660      0.139485                    0.000000   
4677   0.087604  488.589542      0.032122                    0.000000   

      bedwars_bedwars_killer  bedwars_collectors_edition  bedwars_level  \
0                   0.857369                    

In [56]:
# df = pd.get_dummies(df)
# print("dummies")

X = df_scaled.drop('Cheat', axis=1)
y = df_scaled['Cheat']

# データの標準化
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# モデルの作成
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 訓練データの正解率
train_accuracy = model.score(X_train, y_train)
print(f'Training Accuracy: {train_accuracy}')

# テストデータの正解率
test_accuracy = model.score(X_test, y_test)
print(f'Testing Accuracy: {test_accuracy}')

# モデルと標準化器の保存
# joblib.dump(model, './models/Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')


Training Accuracy: 0.9997327632282202
Testing Accuracy: 0.9412393162393162


In [58]:
joblib.dump(model, './models/Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')

['./models/Cheater.pkl']

# snipe垢の条件

fkdr, wlr, bblrは低い

star n.levは30以下

sniper%が高い　（ターゲットが見つかるまで何回もｒｑするから）

fk/lev bb/lev kill/lev が高い　fk/levは7.5 bb/levは6.5 kill/levは20以上で高い

# cheatersデータ - snipe垢 = not sniper cheaters
snipe垢の条件外のcheaterを予測する。

# cubelifyのsniper%
cubelifyのsniper%を取得して、学習データとする